In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn import tree
import warnings
from collections import Counter
import requests
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

C:\Users\Federico\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Leo los datos

In [2]:
df_train=pd.read_csv("entrenamiento.csv", sep=",")
df_lectores=pd.read_csv("lectores.csv",sep=",")
df_libros=pd.read_csv("libros.csv",sep=",")
df_test=pd.read_csv("prueba.csv",sep=",")

## Acomodamos df_libros

In [3]:
#elimino duplicados
df_libros.drop_duplicates(subset='id_libro', inplace=True)

## Web scraping para los libros que no tengo informacion del titulo, genero, etc

In [ ]:
libros_na=df_libros[df_libros["titulo"].isna()]

In [ ]:
def get_book_title(url):
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        title_input = soup.find('input', {'name': 'titulo'})
        if title_input:
            return title_input['value']
        else:
            return "Título no encontrado"
    else:
        return "Error al hacer la solicitud"


def get_book_author(url):
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        author_input = soup.find('input', {'name': 'autor'})
        if author_input:
            return author_input['value']
        else:
            return "Autor no encontrado"
    else:
        return "Error al hacer la solicitud"
    
def get_book_genre(url):
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        genre_input = soup.find('input', {'name': 'genero'})
        if genre_input:
            return genre_input['value']
        else:
            return "Género no encontrado"
    else:
        return "Error al hacer la solicitud"
    
def get_book_editorial(url):
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        # Encontrar el <li> que contiene "Editorial"
        editorial_span = soup.find('span', text='Editorial')
        if editorial_span:
            editorial_li = editorial_span.find_parent('li')
            if editorial_li:
                editorial_a = editorial_li.find('a')
                if editorial_a:
                    return editorial_a.get_text()
                else:
                    return "Editorial no encontrada"
            else:
                return "Editorial no encontrada"
        else:
            return "Editorial no encontrada"
    else:
        return "Error al hacer la solicitud"

def get_book_year(url):
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        # Encontrar el <li> que contiene "Año de edición"
        year_span = soup.find('span', text='Año de edición')
        if year_span:
            year_li = year_span.find_parent('li')
            if year_li:
                year = year_li.get_text(strip=True).replace('Año de edición', '').strip()
                return year
            else:
                return "Año de edición no encontrado"
        else:
            return "Año de edición no encontrado"
    else:
        return "Error al hacer la solicitud"


def add_book_details(df, title_column):
    base_url = "https://quelibroleo.com/"
    titles = []
    authors = []
    genres = []
    editorials = []
    years = []
    
    for title in df[title_column]:
        book_url_fragment = title.lower().replace(' ', '-')
        book_url = base_url + book_url_fragment
        
        book_title = get_book_title(book_url)
        titles.append(book_title)
        
        book_author = get_book_author(book_url)
        authors.append(book_author)
        
        book_genre = get_book_genre(book_url)
        genres.append(book_genre)
        
        book_editorial = get_book_editorial(book_url)
        editorials.append(book_editorial)
        
        book_year = get_book_year(book_url)
        years.append(book_year)
    
    df['titulo'] = titles
    df['autor'] = authors
    df['genero'] = genres
    df['editorial'] = editorials
    df['anio_edicion'] = years
    return df


In [ ]:
libros_na=add_book_details(libros_na, "id_libro")

In [ ]:
## ahora rellenos los Nan en df_libros
df_libros = df_libros.combine_first(libros_na)

In [ ]:
# uno que tiene mal el año lo corrijo a mano
df_libros[df_libros["anio_edicion"]=="(200"]

In [ ]:
df_libros.loc[104, 'anio_edicion'] = 1866

## Nota media de los libros

In [ ]:
def get_book_rating(url):
    res = requests.get(url)
    if res.status_code == 200:
        soup = BeautifulSoup(res.content, 'html.parser')
        estadisticas_div = soup.find(class_="estadisticas")
        rating_span = estadisticas_div.find('span', itemprop="ratingValue")
        if rating_span:
            return rating_span.get_text()
        else:
            return "Puntuación no encontrada"
    else:
        return "Error al hacer la solicitud"

def add_book_ratings(df, title_column):
    base_url = "https://quelibroleo.com/"
    ratings = []
    
    for title in df[title_column]:
        # Convertir el título en una URL amigable
        book_url_fragment = title.lower().replace(' ', '-')
        book_url = base_url + book_url_fragment
        
        # Obtener la puntuación del libro
        rating = get_book_rating(book_url)
        ratings.append(rating)
    
    # Añadir la nueva columna con las puntuaciones al DataFrame
    df['rating'] = ratings
    return df

In [ ]:
df_libros = add_book_ratings(df_libros, 'id_libro')

In [ ]:
### como el scraping tarda mucho dejo lo escrapeado como libros_rankiados.csv
df_libros.to_csv("libros_rankiados.csv", index=False)

In [4]:
df_libros=pd.read_csv("libros_rankiados.csv",sep=",")

In [5]:
df_libros['titulo'] = df_libros['titulo'].str.lower().str.replace(' ', '').str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_libros['autor'] = df_libros['autor'].str.lower().str.replace(' ', '').str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_libros['genero'] = df_libros['genero'].str.lower().str.replace(' ', '').str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_libros['editorial'] = df_libros['editorial'].str.lower().str.replace(' ', '').str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [6]:
df_libros["anio_edicion"] = pd.to_numeric(df_libros["anio_edicion"], errors="coerce")
df_libros["antiguedad_libro"] = np.where(df_libros["anio_edicion"].notnull(), 2024 - df_libros["anio_edicion"], np.nan)

In [7]:
col_eliminar=["resumen","img_src","isbn","anio_edicion"]
df_libros.drop(col_eliminar, axis=1, inplace= True)

### Reduzco la cantidad de autores

In [8]:
cant_librosxautor=df_libros.groupby("autor").id_libro.count().reset_index().set_index("autor").sort_values(by="id_libro",ascending=False)

In [9]:
autores_varios=cant_librosxautor[cant_librosxautor["id_libro"]<15].reset_index()[["autor"]]

In [10]:
autores_varios_set = set(autores_varios['autor'])

# Función para aplicar la lógica de reemplazo
def identificar_autor(autor):
    if autor in autores_varios_set:
        return 'N/s'
    else:
        return autor

# Aplicar la función al DataFrame df_libros
df_libros['autor'] = df_libros['autor'].apply(identificar_autor)

In [11]:
conteo_libros = df_libros.groupby('autor')['id_libro'].count().reset_index()
conteo_libros = conteo_libros.rename(columns={'id_libro': 'cantidad_libros'}).sort_values(by="cantidad_libros")

### Reduzco la cantidad de editoriales

In [12]:
cant_librosxeditorial=df_libros.groupby("editorial").id_libro.count().reset_index().set_index("editorial").sort_values(by="id_libro",ascending=False)

In [13]:
editorial_varios=cant_librosxeditorial[cant_librosxeditorial["id_libro"]<20].reset_index()[["editorial"]]

In [14]:
editorial_varios_set = set(editorial_varios['editorial'])

# Función para aplicar la lógica de reemplazo
def identificar_editorial(editorial):
    if editorial in editorial_varios_set:
        return 'N/s'
    else:
        return editorial

# Aplicar la función al DataFrame df_libros
df_libros['editorial'] = df_libros['editorial'].apply(identificar_editorial)

## Unifico generos 

In [16]:
mapeo_generos = {"narrativahistorica": 'narrativa',
                 'policiaca.novelanegraenbolsillo': 'novelanegra,intriga,terror',
                'novelanegra': 'novelanegra,intriga,terror', 
                 'estudiosyensayos': 'ensayo',
                 "novela": 'literaturacontemporanea',
                 'juvenil': 'infantilyjuvenil',
                 "cienciaficcionenbolsillo": 'fantastica,cienciaficcion',
                 "comicsdehumor": 'comics,novelagrafica',
                 "comics": 'comics,novelagrafica',
                  'romantica': 'romantica,erotica', 
                  'poesia': 'poesia,teatro',
                  'cocina': 'varios',
                 'fotografia': 'varios',
                 'idiomas': 'varios',
                 'politicanacional': 'varios',
                 'guiasdeviaje': 'varios',
                 'television': 'varios',
                 'didacticaymetodologia': 'varios',
                 'deportessobreruedas': 'varios',
                 'albumesilustrados': 'varios',
                 'deportesyjuegos': 'varios',
                 'albumesilustrados': 'varios',
                'informatica': 'varios',
                'cienciaspoliticasysociales': 'varios',
                "naturalezayciencia": 'varios',
                "psicologiaypedagogia": 'varios',
                "filologia": 'varios',
                'derecho': 'varios',
                'naturalezayciencia': 'varios',
                "matematicasdivulgativas": 'varios',
                 'ciencias': 'varios',
                'actores': 'varios',
                'historiadelcine': 'varios',
                 'biografias': 'varios',
                'economiafinanciera': 'varios',
                'economia': 'varios',
                'historiamodernadeespana': 'historia',
                "historica": 'historia',
                'autoayuda': 'autoayudayespiritualidad',
                "astrologiayhoroscopos": 'autoayudayespiritualidad',
                "musica": 'arte',   
                "albumesilustrados"
                "albumesilustrados": 'arte',                  
                'cienciashumanas': 'medicina',
                "medicinadivulgativa": 'medicina',
                "dieteticaynutricion": 'medicina',    
                 'administracionydireccionempresarial': 'empresa'
                }


In [17]:
df_libros['genero'] = df_libros['genero'].replace(mapeo_generos)

# Acomodamos Lectores

In [18]:
df_lectores.rename(columns={'genero':'sexo'}, inplace=True)

In [19]:
df_lectores['sexo'] = df_lectores['sexo'].str.lower()

In [20]:
df_lectores[df_lectores["sexo"].isna()]

,id_lector,nombre,sexo,vive_en,nacimiento
14,15,jose felix,NaN,Santoña - España,NaN
36,37,iván rd,NaN,Toledo - España,NaN
45,46,juan,NaN,España,NaN
90,91,ángela,NaN,España,1910.0
94,95,mª josé lópez-tello,NaN,Málaga - España,1966.0
100,101,ana lopi lopezuela,NaN,España,NaN
101,102,eldominadas,NaN,Oleiros - España,1975.0
108,109,rómulo,NaN,San antonio de los altos - Venezuela,1992.0
115,116,hector tejada marivela,NaN,NaN,NaN
120,121,alicia delgado,NaN,España,NaN


In [21]:
df_lectores.loc[df_lectores["id_lector"] == 15, "sexo"] = "hombre"
df_lectores.loc[df_lectores["id_lector"] == 37, "sexo"]= "hombre"
df_lectores.loc[df_lectores["id_lector"] == 46, "sexo"]= "hombre"
df_lectores.loc[df_lectores["id_lector"] == 91, "sexo"]= "mujer"
df_lectores.loc[df_lectores["id_lector"] == 95, "sexo"]= "mujer"
df_lectores.loc[df_lectores["id_lector"] == 101, "sexo"]= "mujer"
df_lectores.loc[df_lectores["id_lector"] == 102, "sexo"]= "hombre"
df_lectores.loc[df_lectores["id_lector"] == 109, "sexo"]= "hombre"
df_lectores.loc[df_lectores["id_lector"] == 116, "sexo"]= "hombre"
df_lectores.loc[df_lectores["id_lector"] == 121, "sexo"]= "mujer"
df_lectores.loc[df_lectores["id_lector"] == 130, "sexo"]= "mujer"
df_lectores.loc[df_lectores["id_lector"] == 160, "sexo"]= "hombre"
df_lectores.loc[df_lectores["id_lector"] == 174, "sexo"]= "hombre"
df_lectores.loc[df_lectores["id_lector"] == 202, "sexo"]= "mujer"
df_lectores.loc[df_lectores["id_lector"] == 225, "sexo"]= "hombre"

In [22]:
df_lectores.groupby("sexo").nacimiento.mean()

sexo
hombre    1975.895161
mujer     1976.369863
Name: nacimiento, dtype: float64

In [23]:
df_lectores.loc[df_lectores["sexo"] == "hombre", "nacimiento"] = df_lectores.loc[df_lectores["sexo"] == "hombre", "nacimiento"].fillna(1976)
df_lectores.loc[df_lectores["sexo"] == "mujer", "nacimiento"] = df_lectores.loc[df_lectores["sexo"] == "mujer", "nacimiento"].fillna(1976)


In [24]:
df_lectores["nacimiento"] = pd.to_numeric(df_lectores["nacimiento"], errors="coerce")
df_lectores["edad"] = np.where(df_lectores["nacimiento"].notnull(), 2024 - df_lectores["nacimiento"], np.nan)

In [25]:
###una vez que tengo el genero nombre se va
col_eliminar=['nombre','vive_en',"nacimiento"]
df_lectores.drop(col_eliminar, axis=1, inplace= True)

## Filtro lectores que estan en el train pero no en el test

In [26]:
lector_f=df_test['id_lector'].unique()

df_train=df_train.loc[(df_train["id_lector"].isin(lector_f))]

(41277, 3)

## Hago el merge con libros y lectores

In [27]:
df_train_libro=pd.merge(df_train,df_libros,on='id_libro',how='left')
df_test_libro=pd.merge(df_test,df_libros,on='id_libro',how='left')

In [28]:
df_train_libro_lector=pd.merge(df_train_libro,df_lectores,on='id_lector',how='left')
df_test_libro_lector=pd.merge(df_test_libro,df_lectores,on='id_lector',how='left')

## Filtro train con los autores y editoriales que estan en test

In [29]:
autor_f=df_test_libro_lector['autor'].unique()
editorial_f=df_test_libro_lector['editorial'].unique()

df_train_libro_lector=df_train_libro_lector.loc[(df_train_libro_lector["autor"].isin(autor_f)) &
                                              (df_train_libro_lector["editorial"].isin(editorial_f))]

## Que generos le gusta a cada lector?

In [30]:
ranking_genero=df_train_libro_lector.groupby(['id_lector',"genero"])['label'].value_counts().unstack(fill_value=0).reset_index()

In [31]:
ranking_genero["total_leidos"]= ranking_genero[0] + ranking_genero[1]

In [32]:
ranking_genero["prob_like"]=ranking_genero[1]/ranking_genero["total_leidos"]

In [33]:
# Define el término de suavización (alpha)
alpha = 1

# Aplica la suavización de Laplace
ranking_genero["prob_like_smoothed_libro"] = (ranking_genero[1] + alpha) / (ranking_genero["total_leidos"] + 2 * alpha)

In [34]:
ranking_genero=ranking_genero[["id_lector","genero","prob_like_smoothed_libro"]]

In [35]:
prob_like_genero=pd.pivot_table(ranking_genero,values="prob_like_smoothed_libro",index="id_lector",columns="genero").fillna(0).reset_index()

In [36]:
libros_entrenar=df_train_libro_lector[["id_lector","id_libro","genero"]]
libros_predecir=df_test_libro_lector[["id_lector","id_libro","genero"]]

In [37]:
perfil_lector_entrenar=prob_like_genero.merge(libros_entrenar,on="id_lector",how="inner")
perfil_lector_predecir=prob_like_genero.merge(libros_predecir,on="id_lector",how="inner")

In [38]:
def le_gustara_genero(df):
    df['le va a gustar/segun_genero'] =0
    def asignar_probabilidad(row):
        genero = row['genero']  
        probabilidad = row[genero]
        return probabilidad
    df['le va a gustar/segun_genero'] = df.apply(asignar_probabilidad, axis=1)
    return df[["id_libro","id_lector",'le va a gustar/segun_genero']]

In [39]:
le_va_a_gustar_entrenar = le_gustara_genero(perfil_lector_entrenar)
le_va_a_gustar_predecir = le_gustara_genero(perfil_lector_predecir)

In [40]:
df_train_libro_lector=pd.merge(df_train_libro_lector,le_va_a_gustar_entrenar,on=["id_libro","id_lector"],how="left")
df_test_libro_lector=pd.merge(df_test_libro_lector,le_va_a_gustar_predecir,on=["id_libro","id_lector"],how="left")

# Armo el index

In [41]:
df_train_libro_lector['id_lector'] = df_train_libro_lector['id_lector'].mask(lambda x: x.notna(), df_train_libro_lector['id_lector'].astype(str))
df_test_libro_lector['id_lector'] = df_test_libro_lector['id_lector'].mask(lambda x: x.notna(), df_test_libro_lector['id_lector'].astype(str))

In [42]:
df_train_libro_lector['index'] = df_train_libro_lector['id_lector'] + "--" + df_train_libro_lector['id_libro']

# Paso 2: Establecer la nueva columna como índice
df_train_libro_lector.set_index('index', inplace=True)

df_test_libro_lector['index'] = df_test_libro_lector['id_lector'] + "--" + df_test_libro_lector['id_libro']

# Paso 2: Establecer la nueva columna como índice
df_test_libro_lector.set_index('index', inplace=True)

In [43]:
col_eliminar=["id_libro","titulo"]
df_train_libro_lector.drop(col_eliminar, axis=1, inplace= True)
df_test_libro_lector.drop(col_eliminar, axis=1, inplace= True)

In [44]:
df_train_libro_lector=pd.get_dummies(df_train_libro_lector, columns=['sexo',"autor","editorial",'genero']) # , dummy_na=True
df_test_libro_lector=pd.get_dummies(df_test_libro_lector, columns=['sexo',"autor","editorial",'genero']) # , dummy_na=True

In [45]:
# convertir ranking a entero
df_train_libro_lector['rating'] = df_train_libro_lector['rating'].str.replace("'", "")
df_train_libro_lector['rating'] = df_train_libro_lector['rating'].str.replace(",", ".")
df_train_libro_lector['rating'] = pd.to_numeric(df_train_libro_lector['rating'], errors='coerce')

## Eliminar NaNs
df_train_libro_lector['rating'].fillna(2, inplace=True, downcast= "infer")

# Convertir la columna 'rating' a int
df_train_libro_lector['rating'] = df_train_libro_lector['rating'].astype(float)

# Procesamiento para df_test_libro_lector
df_test_libro_lector['rating'] = df_test_libro_lector['rating'].str.replace("'", "")
df_test_libro_lector['rating'] = df_test_libro_lector['rating'].str.replace(",", ".")
df_test_libro_lector['rating'] = pd.to_numeric(df_test_libro_lector['rating'], errors='coerce')

# Completar NaN
df_test_libro_lector['rating'].fillna(2, inplace=True, downcast= "infer")
df_test_libro_lector['rating'] = df_test_libro_lector['rating'].astype(float)

In [46]:
df_train_libro_lector["le va a gustar/segun_genero"]=df_train_libro_lector["le va a gustar/segun_genero"]*10
df_test_libro_lector["le va a gustar/segun_genero"]=df_test_libro_lector["le va a gustar/segun_genero"]*10
df_train_libro_lector["le va a gustar/segun_genero"]=df_train_libro_lector["le va a gustar/segun_genero"].round(2)
df_test_libro_lector["le va a gustar/segun_genero"]=df_test_libro_lector["le va a gustar/segun_genero"].round(2)


In [47]:
# La creación de modelos requiere que no haya valores perdidos
df_train_libro_lector.fillna(0, inplace=True, downcast= "infer")
df_test_libro_lector.fillna(0, inplace=True, downcast= "infer")

# 3. Entrenamiento del modelos (AA) -- ⛔⛔⛔ NO TOCAR ⛔⛔⛔

In [ ]:
def entrenamiento(df):
    df = df.select_dtypes(include=['float64', 'int64', 'int32', 'int16', 'int8', 'bool'])

    X = df[df.columns.drop('label')]
    y = df['label']
    
    for n_estimators in [50, 100, 500, 1000]:
        for max_depth in [5, 10, 15, 30]:
            print(f"{n_estimators=} -- {max_depth=}")

            # Creamos el modelo
            reg = sk.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,n_jobs=-1, random_state=42)

            scores_train = []
            scores_test = []

            # Validación cruzada, 10 folds, shuffle antes, semilla aleatoria
            kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)

            for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
                # Partimos el fold en entrenamiento y prueba...
                X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

                # Entrenamos el modelo en entramiento
                reg.fit(X_train, y_train)

                # Predecimos en train
                y_pred = reg.predict(X_train)

                # Medimos la performance de la predicción en entramiento
                score_train = sk.metrics.f1_score(y_train, y_pred)
                scores_train.append(score_train)

                # Predecimos en test
                y_pred = reg.predict(X_test)

                # Medimos la performance de la predicción en prueba
                score_test = sk.metrics.mean_squared_error(y_test, y_pred, squared=False)
                scores_test.append(score_test)

                print("\t", f"{fold=}, {score_train=} {score_test=}")

            print(f"Media de scores en entrenamiento={pd.Series(scores_train).mean()}, std={pd.Series(scores_train).std()}")
            print(f"Media de scores en prueba={pd.Series(scores_test).mean()}, std={pd.Series(scores_test).std()}")
            print()

In [ ]:
entrenamiento(df_train_libro_lector)

# 4. Predicción para kaggle -- ⚠️⚠️⚠️ MODIFICAR HIPERPARÁMETROS ⚠️⚠️⚠️

In [48]:
def predecir(df_train,df_test,n_estimators,max_depth):
    ## Datos a predecir
    X = df_train[df_train.columns.drop('label')]
    y = df_train['label']

    X_test = df_test[df_train.columns.drop('label')]

    reg = sk.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)
    reg.fit(X, y)

    # Predecimos
    df_test['label'] = reg.predict(X_test)

    # Creamos el dataframe para entregar
    df_sol = df_test[["label"]]
    return df_sol,reg,X

In [49]:
predic,mod,X=predecir(df_train_libro_lector,df_test_libro_lector,1000,30)

In [50]:
predic=predic.rename_axis('id')

In [51]:
# Tests de validación de la predicción antes de subirla
# Estos tests TIENEN que pasar sin error

assert predic.shape[0] == 10332, f"La cantidad de filas no es correcta. Es {predic.shape[0]} y debe ser 10332."
assert predic.shape[1] == 1, f"La cantidad de columnas no es correcta. Es {predic.shape[1]} y debe ser 1."
assert 'label' in predic.columns, "Falta la columna 'price'."
assert predic.index.name == 'id', "El índice debe llamarse 'id'."

In [52]:
# Grabamos el archivo para subir a kaggle
# TODO: Cambiar la versión y llevar registro

version = "vFinal"
predic.to_csv(f"solucion-{version}.csv", index=True)